# Практика

In [1]:
import pandas as pd
import numpy as np
import pickle
import random

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

In [2]:
from pathlib import Path


DATA_ROOT = Path('C:/Users/Sancho/Downloads/python 2/lesson 3/')
MODELS_PATH = Path('./models/')

# input
DATASET_PATH = DATA_ROOT / 'train.csv'

# output
TRAIN_FULL_PATH = DATA_ROOT / 'training_project_train_full.csv'
TRAIN_PART_PATH = DATA_ROOT / 'training_project_train_part_b.csv'
TEST_PART_PATH = DATA_ROOT / 'training_project_test_part.csv'

In [3]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [4]:
def evaluate_preds(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

In [5]:
df = pd.read_csv(DATASET_PATH)
df.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [6]:
TARGET_NAME = 'Credit Default'
BASE_FEATURE_NAMES = df.columns.drop(TARGET_NAME).tolist()

In [7]:
NUM_FEATURE_NAMES = ['Annual Income','Tax Liens','Number of Open Accounts','Years of Credit History','Maximum Open Credit',
                     'Number of Credit Problems','Months since last delinquent','Bankruptcies','Current Credit Balance',
                     'Monthly Debt','Credit Score']

In [8]:
class DataCorrection:
    def __init__(self, path_to_file):
        
        self.add_dummies_columns = None
        self.df = pd.read_csv(path_to_file)
        self.medians=None
        
    def fit(self, df):
        self.medians = df.median()
    
    def fill(self):
        df = self.df.copy()
        add_dummies_columns = None

        Term_to_nubmers = {'Short Term':'0', 'Long Term':'1'}
        
        Purpose_to_numbers = {'debt consolidation':'0', 'other':'1', 'home improvements':'2', 'business loan':'3',
                              'buy a car':'3', 'medical bills':'3', 'major purchase':'3', 'take a trip':'1',
                              'buy house':'1', 'small business':'1', 'wedding':'1', 'moving':'1',
                              'educational expenses':'1', 'vacation':'1', 'renewable energy':'1'}
        
        Home_O_to_numbers = {'Home Mortgage':'0', 'Rent':'1', 'Own Home':'2', 'Have Mortgage':'0'}
        
        Job_Years_to_numbers = {'< 1 year':'0',
                            '1 year':'1','2 years':'1','3 years':'1','4 years':'2','5 years':'2', 
                            '6 years':'2','7 years':'3','8 years':'3','9 years':'3',
                            '10+ years':'4'}
        
        df.loc[df['Credit Score'] >= 3000, 'Credit Score'] = (df['Credit Score'] / 10)
        df.loc[df['Current Loan Amount'] >= 9000000.0, 'Current Loan Amount'] = self.medians['Current Loan Amount']
        
        df['Annual Income'].fillna(self.medians['Annual Income'], inplace=True)
        df['Home Ownership'].fillna('Rent', inplace=True)
        df['Years in current job'].fillna('2 years', inplace=True)
        df['Bankruptcies'].fillna(0.0, inplace=True)
        df['Months since last delinquent'].fillna(self.medians['Months since last delinquent'], inplace=True)
        df['Credit Score'].fillna(self.medians['Credit Score'], inplace=True)
        df.loc[df['Bankruptcies'] >= 3, 'Bankruptcies'] = 3
        df['Years in current job'] = df['Years in current job'].map(Job_Years_to_numbers)
        df['Home Ownership'] = df['Home Ownership'].map(Home_O_to_numbers)
        df['Purpose'] = df['Purpose'].map(Purpose_to_numbers)
        df['Term'] = df['Term'].map(Term_to_nubmers)
        
        return df

In [9]:
correction = DataCorrection(DATASET_PATH)
correction.fit(df)
train_df=correction.fill()

In [12]:
scaler = StandardScaler()

df_norm = train_df.copy()
df_norm[NUM_FEATURE_NAMES] = scaler.fit_transform(df_norm[NUM_FEATURE_NAMES])

train_df = df_norm.copy()

In [13]:
X = train_df[BASE_FEATURE_NAMES]
y = train_df[TARGET_NAME]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    shuffle=True,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=y)

display(y_train.value_counts(normalize=True), y_test.value_counts(normalize=True))

0    0.718286
1    0.281714
Name: Credit Default, dtype: float64

0    0.718222
1    0.281778
Name: Credit Default, dtype: float64

In [14]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [15]:
df.to_csv(TRAIN_FULL_PATH, index=False, encoding='utf-8')
train.to_csv(TRAIN_PART_PATH, index=False, encoding='utf-8')
test.to_csv(TEST_PART_PATH, index=False, encoding='utf-8')

In [16]:
model_tree = DecisionTreeClassifier(random_state=21,
                                    class_weight={0:1, 1:3.6},
                                    max_depth=4
                                    )
model_tree.fit(X_train, y_train)

evaluate_preds(model_tree, X_train, X_test, y_train, y_test)

TRAIN

              precision    recall  f1-score   support

           0       0.87      0.36      0.51      3771
           1       0.35      0.87      0.50      1479

    accuracy                           0.50      5250
   macro avg       0.61      0.61      0.50      5250
weighted avg       0.73      0.50      0.51      5250

TEST

              precision    recall  f1-score   support

           0       0.85      0.36      0.51      1616
           1       0.34      0.84      0.49       634

    accuracy                           0.50      2250
   macro avg       0.60      0.60      0.50      2250
weighted avg       0.71      0.50      0.50      2250

CONFUSION MATRIX

col_0             0     1
Credit Default           
0               588  1028
1               100   534


In [18]:
%%time
model_catb = catb.CatBoostClassifier(silent=True, random_state=21)
model_catb.fit(X_train, y_train)

evaluate_preds(model_catb, X_train, X_test, y_train, y_test)

TRAIN

              precision    recall  f1-score   support

           0       0.84      0.99      0.91      3771
           1       0.93      0.51      0.66      1479

    accuracy                           0.85      5250
   macro avg       0.89      0.75      0.78      5250
weighted avg       0.86      0.85      0.84      5250

TEST

              precision    recall  f1-score   support

           0       0.75      0.93      0.83      1616
           1       0.54      0.21      0.30       634

    accuracy                           0.73      2250
   macro avg       0.64      0.57      0.57      2250
weighted avg       0.69      0.73      0.68      2250

CONFUSION MATRIX

col_0              0    1
Credit Default           
0               1501  115
1                501  133
Wall time: 4 s


# Текстовая часть

In [1]:
# 1.Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?
# Weighted - вычисляет F1 для каждого класса отдельно, но при сложении этих значений каждое из них умножается на вес 
# класса который зависит от количества положительных результатов, тем самым отдавая предпочтение большим классам.
# Macro - также вычилсяет F1 для каждого класса отдельно, но уже не добавляет вес к слагаемым, тем самым увеличивая 
# "наказание" за ошибку в малых классах 
# Micro - подсчитывает данные F1 (TP, FN, FP) для всех классов одноверменно не отдавая предпочтение ни одному из классов
# Macro можно использовать при наличии больших и не менее важных малых выборок, к примеру как в курсовой, к которой
# мы готовимся. Weighted будет использоваться для увеличения точности в нескольких основных классах, жертвуя точностью
# оставшихся малых. 

In [ ]:
# 2.В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?
# xgboost использует похожесть полученных "листочков" как аналог энтропии Шеннона для вычисления прироста инфрмации от 
# каждого производимого разбиения и задавая гиперпараметр лямбду мы можем настроить минимальный прирост информации от
# производимых разбиений и тем самым улучшить защиту от переобучения. Из трех представленных, как мне показалось,
# этот метод ближе всех к классическому градиентному бустингу.
# Lightgbm - придумывался как более быстрая версия градиентного бустинга, где идет выборка по строкам и уменьшение(объединение)
# признаков. Также он отличается способом построения дерева. В этом методе построение идет от "листочков" с 
# наибольшей ошибкой для быстрого уменьшения кол-ва ошибок
# Особенность catboost в симметричности деревьев и за счет этого скорость обучения увеличивается в разы. Также у этой
# модели есть встроенное предотвращение переобучения в виде перемешивания выборки и обучения нескольких моделей одновременно,
# которые будут проверяться на данных которые они до этого не видели